# Books Data Collection

In [ ]:
from flask import Flask, request, render_template, jsonify
import requests
import psycopg2
import time

In [ ]:
DB_HOST = "isrpeer.ctw842oc69v3.us-east-2.rds.amazonaws.com"
DB_NAME = "isr_books"
DB_USER = "isr_peer"
DB_PASS = "ISRpeer2024"

def connect():
    conn_string = "host="+ DB_HOST +" port="+ "5432" +" dbname="+ DB_NAME +" user=" + DB_USER+" password="+ DB_PASS
    conn = psycopg2.connect(conn_string)
    print("Connected!")
    cursor = conn.cursor()
    return conn, cursor

In [ ]:
conn, cursor = connect()

Connected!


In [ ]:
# Creating Data Table 'new_merged_books' in the database
def create_table(DB_HOST, DB_NAME, DB_USER, DB_PASS, cursor):

    # SQL command to create a table
    create_table_command = """
    CREATE TABLE IF NOT EXISTS new_merged_books (
        id SERIAL PRIMARY KEY,
        Title VARCHAR(255),
        Author TEXT,
        Publisher VARCHAR(255),
        PublishedDate VARCHAR(20),
        Description TEXT,
        PrintType VARCHAR(50),
        AverageRating REAL,
        Category TEXT,
        MaturityRating VARCHAR(50),
        InfoLink TEXT,
        IndustryIdentifiers TEXT,
        ReadingModes TEXT,
        PageCount INT,
        RatingsCount INT,
        ImageLinks TEXT,
        PreviewLink TEXT,
        Language VARCHAR(10),
        ListPrice REAL,
        RetailPrice REAL,
        BuyLink TEXT
    );
    """

    # Execute the SQL command
    cursor.execute(create_table_command)
    conn.commit()  # Commit the changes to the database


    print("Table created successfully")

create_table(DB_HOST, DB_NAME, DB_USER, DB_PASS, cursor)

In [ ]:
# Data Retrieval from Google Books API
# Function to extract and insert book data
def extract_and_insert_book_data(book, cursor):
    volume_info = book.get('volumeInfo', {})
    sale_info = book.get('saleInfo', {})
    book_data = (
        volume_info.get('title'),
        ', '.join(volume_info.get('authors', [])),
        volume_info.get('publisher'),
        volume_info.get('publishedDate'),
        volume_info.get('description'),
        volume_info.get('printType'),
        volume_info.get('averageRating'),
        ', '.join(volume_info.get('categories', [])),
        volume_info.get('maturityRating'),
        volume_info.get('infoLink'),
        ', '.join([f"{id['type']}: {id['identifier']}" for id in volume_info.get('industryIdentifiers', [])]),
        str(volume_info.get('readingModes', {})),
        volume_info.get('pageCount'),
        volume_info.get('ratingsCount'),
        ', '.join(volume_info.get('imageLinks', {}).values()),
        volume_info.get('previewLink'),
        volume_info.get('language'),
        sale_info.get('listPrice', {}).get('amount'),
        sale_info.get('retailPrice', {}).get('amount'),
        sale_info.get('buyLink')
    )
    insert_query = '''
    INSERT INTO new_merged_books (
        Title, Author, Publisher, PublishedDate, Description, PrintType, AverageRating,
        Category, MaturityRating, InfoLink, IndustryIdentifiers, ReadingModes, PageCount,
        RatingsCount, ImageLinks, PreviewLink, Language, ListPrice, RetailPrice, BuyLink
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''
    cursor.execute(insert_query, book_data)

# Function to search and store books
def search_books(query, max_books=4000):
    #conn = connect()
    print("after connect")
    #cursor = conn.cursor()
    api_key = "Your API Key"
    books_data = []
    inserted = 0
    start_index = 0
    max_results = 10

    total_books_to_fetch = 100


    while start_index < total_books_to_fetch:
      api_url = f'https://www.googleapis.com/books/v1/volumes?q={query}&maxResults={max_results}&startIndex={start_index}&key={api_key}'
      response = requests.get(api_url)
      data = response.json()
      books = data.get('items', [])
      if not books:
        break
      books_data.extend(books)
      for book in books:
        extract_and_insert_book_data(book, cursor)
      conn.commit()
      start_index += 1



search_books("Search Query")

In [ ]:
#Complexity Level Classification
import psycopg2
from textstat import textstat

# Define the keywords for classification
keywords = {
    'Beginner': ['introduction', 'fundamentals', 'basics', 'beginner', 'overview', '101', 'introductory', 'start', 'for beginners', 'no prior knowledge', 'essential', 'first steps'],
    'Intermediate': ['intermediate', 'beyond basics', 'develop skills', 'enhanced', 'expanded', 'practical application', 'skills improvement', 'next level', 'deepen', 'more depth', 'comprehensive'],
    'Advanced': ['advanced', 'expert', 'mastery', 'specialized', 'complex', 'in-depth', 'intensive', 'advanced techniques', 'professional', 'expertise']
}

# Complexity calculation functions
def classify_by_keywords(text, keywords):
    text = text.lower()
    for level, keys in keywords.items():
        if any(key in text for key in keys):
            return level
    return 'Unknown'

def classify_by_flesch(flesch_score):
    if flesch_score >= 70:
        return 'Beginner'
    elif 50 <= flesch_score < 70:
        return 'Intermediate'
    else:
        return 'Advanced'

def calculate_complexity(title, description):
    full_text = (title + " " + description).strip()
    complexity = classify_by_keywords(full_text, keywords)
    if complexity == 'Unknown':
        flesch_score = textstat.flesch_reading_ease(full_text)
        complexity = classify_by_flesch(flesch_score)
    return complexity


In [ ]:
#Adding Complexity Column to existing table 'books'
def add_complexity_column(cursor):
    add_column_command = """
    ALTER TABLE new_merged_books
    ADD COLUMN IF NOT EXISTS Complexity VARCHAR(50);
    """
    cursor.execute(add_column_command)
    conn.commit()
    print("Complexity column added successfully")

def update_book_complexity(cursor):
    # Fetch all book titles and descriptions
    cursor.execute("SELECT id, Title, Description FROM books;")
    books = cursor.fetchall()

    for book_id, title, description in books:
        title = title if title else ""
        description = description if description else ""
        complexity = calculate_complexity(title, description)
        update_query = """
        UPDATE new_merged_books
        SET Complexity = %s
        WHERE id = %s;
        """
        cursor.execute(update_query, (complexity, book_id))
    conn.commit()
    print("Complexity updated for all books")

add_complexity_column(cursor)
update_book_complexity(cursor)

# Online Courses Data Collection

In [ ]:
# Creating the 'online_courses' table in the database
from flask import Flask, request, render_template, jsonify
import requests
import psycopg2
import operator
from requests.auth import HTTPBasicAuth
from bs4 import BeautifulSoup
import pandas as pd

DB_HOST = "isrpeer.ctw842oc69v3.us-east-2.rds.amazonaws.com"
DB_NAME = "isr_books"
DB_USER = "isr_peer"
DB_PASS = "ISRpeer2024"

def connect():
    conn_string = f"host={DB_HOST} dbname={DB_NAME} user={DB_USER} password={DB_PASS}"
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    return conn, cursor

def create_online_courses_table(cursor):
    create_table_command = """
    CREATE TABLE IF NOT EXISTS new_merged_course (
        id SERIAL PRIMARY KEY,
        title VARCHAR(255),
        url TEXT,
        description TEXT,
        headline TEXT,
        num_subscribers INT,
        avg_rating REAL,
        num_reviews INT,
        published_title TEXT,
        primary_category VARCHAR(255),
        primary_subcategory VARCHAR(255),
        num_quizzes INT,
        num_lectures INT,
        num_curriculum_items INT,
        visible_instructors TEXT[],
        is_paid BOOLEAN,
        price VARCHAR(50),
        what_you_will_learn TEXT[],
        who_should_attend TEXT[],
        image_480x270 TEXT,
        image_50x50 TEXT
    );
    """
    cursor.execute(create_table_command)
    conn.commit()
    print("Online courses table created successfully")

create_online_courses_table(cursor)

In [ ]:
# Extracting Data from Udemy API
def extract_and_insert_course_data(course, cursor, conn):
    # Define default values or handle missing keys gracefully
    title = course.get('title', 'N/A')
    url = course.get('url', 'N/A')
    description = course.get('description', 'N/A')
    headline = course.get('headline', 'N/A')
    num_subscribers = course.get('num_subscribers', 0)
    avg_rating = course.get('avg_rating', 0.0)
    num_reviews = course.get('num_reviews', 0)
    published_title = course.get('published_title', 'N/A')
    primary_category = course.get('primary_category', 'N/A')
    primary_subcategory = course.get('primary_subcategory', 'N/A')
    num_quizzes = course.get('num_quizzes', 0)
    num_lectures = course.get('num_lectures', 0)
    num_curriculum_items = course.get('num_curriculum_items', 0)
    visible_instructors = [instructor.get('name', 'N/A') for instructor in course.get('visible_instructors', [])]
    is_paid = course.get('is_paid', False)
    price = course.get('price', 'N/A')
    what_you_will_learn = course.get('what_you_will_learn_data', {}).get('items', [])
    who_should_attend = course.get('who_should_attend_data', {}).get('items', [])
    image_480x270 = course.get('image_480x270', 'N/A')
    image_50x50 = course.get('image_50x50', 'N/A')

    course_data = (
        title, url, description, headline, num_subscribers, avg_rating, num_reviews,
        published_title, primary_category, primary_subcategory, num_quizzes, num_lectures,
        num_curriculum_items, visible_instructors, is_paid, price, what_you_will_learn,
        who_should_attend, image_480x270, image_50x50
    )

    insert_query = '''
    INSERT INTO new_merged_course (
        title, url, description, headline, num_subscribers, avg_rating, num_reviews,
        published_title, primary_category, primary_subcategory, num_quizzes, num_lectures,
        num_curriculum_items, visible_instructors, is_paid, price, what_you_will_learn,
        who_should_attend, image_480x270, image_50x50
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''
    cursor.execute(insert_query, course_data)
    conn.commit()

class Udemy(object):
    __BASE_URL = "https://www.udemy.com/api-2.0/"

    def __init__(self, client_id: str, client_secret: str) -> None:
        self.__client_id = client_id
        self.__client_secret = client_secret

    @property
    def url(self) -> str:
        return self.__BASE_URL

    @property
    def client_id(self):
        return self.__client_id

    @property
    def client_secret(self) -> str:
        return self.__client_secret

    def _get_full_url(self, resource: str, **kwargs) -> str:
        url = f"{self.url}{resource}/?"
        field_string = ""

        if "search" in kwargs:
            url += f"search={kwargs['search']}&"
            del kwargs['search']

        for param, value in sorted(kwargs.items(), key=operator.itemgetter(0)):
            if param != "fields":
                if "category" in param and "&" in value:
                    value = value.replace(" & ", "+%26+")
                url += f"{param}={value}&"
            else:
                for ele in value:
                    object_name = ele["Object"]
                    params = ",".join(
                        filter(
                            None,
                            [
                                ele["Setting"],
                                ",".join(ele["Additions"]),
                                ",".join(["-" + x for x in ele["Minus"]]),
                            ],
                        )
                    )
                    field_string += f"fields[{object_name}]={params}&"
        url += field_string
        return url

    @property
    def _authentication(self) -> HTTPBasicAuth:
        return HTTPBasicAuth(self.client_id, self.client_secret)

    def courses(self, search_term=None, page=1, page_size=10, **kwargs):
        kwargs['page'] = page
        kwargs['page_size'] = page_size

        if search_term:
            kwargs['search'] = search_term

        full_url = self._get_full_url("courses", **kwargs)
        response = requests.get(full_url, auth=self._authentication)
        return response.json()

client_id = "Your Client ID"
client_secret = "Your Client Secret"
udemy = Udemy(client_id, client_secret)

def search_and_store_courses(search_term, max_courses=1000):
    courses_collected = 0
    page_number = 1
    all_courses_data = []

    while courses_collected < max_courses:
        page_size = 100
        courses_response = udemy.courses(search_term=search_term, page=page_number, page_size=page_size)
        courses = courses_response.get('results', [])

        if not courses:
            break

        for course in courses:
            if courses_collected >= max_courses:
                break
            extract_and_insert_course_data(course, cursor, conn)
            courses_collected += 1

        page_number += 1

search_and_store_courses("Search Query")

In [ ]:
# Complexity Level Classification for online_courses
from textstat import textstat

# Updated keywords for classification
keywords = {
    'Beginner': [
        'introduction', 'beginner', 'intro', 'basics', 'fundamentals', 'first steps',
        'novice', 'entry level', 'starter', 'for newbies', 'basic understanding',
        'no experience required', 'zero to hero', 'essentials', '101'
    ],
    'Intermediate': [
        'intermediate', 'next steps', 'beyond basics', 'improve', 'building skills',
        'skill enhancement', 'expand knowledge', 'hands-on learning', 'practical',
        'deep dive', 'more complex', 'detailed', 'proficiency'
    ],
    'Advanced': [
        'advanced', 'expert level', 'masterclass', 'advanced techniques', 'specialization',
        'in-depth', 'high-level', 'complex', 'intensive', 'for professionals', 'expertise',
        'thorough understanding', 'comprehensive coverage', 'deep understanding'
    ]
}

# Complexity calculation functions remain unchanged
def classify_by_keywords(text, keywords):
    text = text.lower()
    for level, keys in keywords.items():
        if any(key in text for key in keys):
            return level
    return 'Unknown'

def classify_by_flesch(flesch_score):
    if flesch_score >= 70:
        return 'Beginner'
    elif 50 <= flesch_score < 70:
        return 'Intermediate'
    else:
        return 'Advanced'

def calculate_complexity(title, headline, description, what_you_will_learn, who_should_attend):
    # Combine the strings with a space to ensure separation between content from different fields
    full_text = " ".join([title, headline, description, what_you_will_learn, who_should_attend]).strip()
    complexity = classify_by_keywords(full_text, keywords)
    if complexity == 'Unknown':
        flesch_score = textstat.flesch_reading_ease(full_text)
        complexity = classify_by_flesch(flesch_score)
    return complexity

def update_online_complexity(cursor):
    cursor.execute("SELECT id, title, headline, description, what_you_will_learn, who_should_attend FROM new_merged_course;")
    books = cursor.fetchall()

    for book_id, title, headline, description, what_you_will_learn, who_should_attend in books:
        title = title if title else ""
        headline = headline if headline else ""
        description = description if description else ""
        what_you_will_learn = what_you_will_learn if what_you_will_learn else ""
        who_should_attend = who_should_attend if who_should_attend else ""

        # Calculate complexity based on combined text fields
        complexity = calculate_complexity(title, headline, description, what_you_will_learn, who_should_attend)
        update_query = """
        UPDATE new_merged_course
        SET Complexity = %s
        WHERE id = %s;
        """
        cursor.execute(update_query, (complexity, book_id))
    conn.commit()
    print("Complexity updated for all new_merged_course")

def add_complexity_column_online(cursor):
    add_column_command = """
    ALTER TABLE new_merged_course
    ADD COLUMN IF NOT EXISTS Complexity VARCHAR(50);
    """
    cursor.execute(add_column_command)
    conn.commit()
    print("Complexity column added successfully")

add_complexity_column_online(cursor)
update_online_complexity(cursor)

# Tokenization for Full text search

In [ ]:
# books table
cursor.execute("""
    ALTER TABLE new_merged_books ADD COLUMN ts tsvector
    GENERATED ALWAYS AS (
        setweight(to_tsvector('english', coalesce(title, '')), 'A') ||
        setweight(to_tsvector('english', coalesce(description, '')), 'B')
    ) STORED;
""")
conn.commit()
print("done")



# online courses table
cursor.execute("""
    ALTER TABLE new_merged_course ADD COLUMN ts tsvector
    GENERATED ALWAYS AS (
        setweight(to_tsvector('english', coalesce(title, '')), 'A') ||
        setweight(to_tsvector('english', coalesce(headline, '')), 'A') ||
        setweight(to_tsvector('english', coalesce(primary_category, '')), 'B') ||
        setweight(to_tsvector('english', coalesce(primary_subcategory, '')), 'B') ||
        setweight(to_tsvector('english', coalesce(description, '')), 'C')
    ) STORED;
""")

conn.commit()
print("done")